In [3]:
!pip install requests pandas python-dotenv

In [4]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os

# Cargar las variables de entorno desde el archivo .env
load_dotenv()

# Obtener las claves API del archivo .env
news_api_key = os.getenv("NEWS_API_KEY")
weather_api_key = os.getenv("WEATHER_API_KEY")
exchange_api_key = os.getenv("EXCHANGE_API_KEY")

# Función para obtener noticias
def get_news(api_key):
    url = f'https://newsapi.org/v2/top-headlines?country=us&apiKey={api_key}'
    response = requests.get(url)
    news_data = response.json()
    
    # Extracción de los títulos y descripciones
    articles = [{'title': article['title'], 'description': article['description']} for article in news_data['articles']]
    
    return pd.DataFrame(articles)

# Función para obtener la temperatura
def get_temperature(api_key, city="London"):
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'
    response = requests.get(url)
    weather_data = response.json()
    
    # Convertir la temperatura de Kelvin a Celsius
    temperature = weather_data['main']['temp'] - 273.15
    return temperature

# Función para obtener el tipo de cambio
def get_exchange_rate(api_key, base="USD", target="EUR"):
    url = f'https://v6.exchangerate-api.com/v6/{api_key}/latest/{base}'
    response = requests.get(url)
    exchange_data = response.json()
    
    # Obtener la tasa de cambio para la moneda objetivo
    exchange_rate = exchange_data['conversion_rates'][target]
    return exchange_rate

# Obtener los datos
news_df = get_news(news_api_key)
temperature = get_temperature(weather_api_key)
exchange_rate = get_exchange_rate(exchange_api_key)

# Mostrar los resultados
print(f"Temperatura en la ciudad: {temperature:.2f} °C")
print(f"Tipo de cambio (USD a EUR): {exchange_rate:.2f}")
print(news_df.head())  # Mostrar las primeras noticias

# Limpiar los datos de noticias (eliminar filas sin título o descripción)
news_df_cleaned = news_df.dropna(subset=['title', 'description'])

# Validación de la temperatura (debe estar dentro de un rango lógico)
if temperature < -50 or temperature > 50:
    print("Temperatura no válida")
else:
    print(f"Temperatura válida: {temperature:.2f} °C")

# Validación del tipo de cambio (debe estar dentro de un rango esperado)
if exchange_rate < 0.5 or exchange_rate > 2:
    print("Tipo de cambio fuera de rango esperado")
else:
    print(f"Tipo de cambio válido: {exchange_rate:.2f}")


Temperatura en la ciudad: 9.61 °C
Tipo de cambio (USD a EUR): 0.91
                                               title  \
0  Hungary will withdraw from ICC, government say...   
1  Dow futures fall after tariffs push market to ...   
2  Senate Republicans vote to advance a massive b...   
3  At first blush, Geno Smith’s contract isn’t dr...   
4  Zoe Saldaña Shows Off ‘Avatar: Fire And Ash’ A...   

                                         description  
0  Hungary announced it would leave the Internati...  
1  The three major indexes are on track to end th...  
2  The Republican-led Senate voted 52-48 on Thurs...  
3  Remember when the Seahawks pivoted from Geno S...  
4  James Cameron's Avatar: Fire and Ash showed of...  
Temperatura válida: 9.61 °C
Tipo de cambio válido: 0.91
